# Kleisli Categories

_P/42 Writer<T> c++ template_

we have accomplished our goal: the aggregation of the log
is no longer the concern of the individual functions

they produce their own messages, which are then, externally
concatenated into a larger log

we have to **abstract function composition**
   
   

## The Writer Category

MY NOTE:

what are in the writer category? a bunch of objects called "Writers" and their morphisms:

$Writer \rightarrow Writer'$



_P/45 composing Writer<T> template: a category point of view_
    
what is the identity morphism (in the Writer category) ?
    
an identity morphism (in the case of `Writer<T>`) should
pass its argument without change, and only contribute an
empty string to the log

**the only requirement is that the `log` type have monoidal
properties**

In [41]:
import Control.Monad

-- the object
type Writer a = (a, String)

-- the morphism
f :: a -> Writer b
f = undefined

-- the new compose operator, the "fish" operator
(>=>) :: (a -> Writer b) -> (b -> Writer c) -> (a -> Writer c)
m1 >=> m2 = \x -> let (o1, state1) = m1 x
                      (o2, state2) = m2 o1
                  in (o2, state1 <> state2)

ident :: Int -> Writer Int
ident x = (x, "called identity morphism\n")

add :: Int -> Int -> Writer Int
add n x = (n + x, "called add with n\n")

isEven :: Int -> Writer Bool
isEven x = (even x, "called isEven\n")

runTests :: IO ()
runTests = do
    let f = ident >=> add 10 >=> isEven
    let (o, log) = f 1
    mapM_ print $ lines log
    
runTests

"called identity morphism"
"called add with n"
"called isEven"